# Classes in Python
...and a bit in general, for those of you who haven't done any object-oriented programming yet.

## What is a class?

Many things :-/

There are several ways to explain classes, some rather abstract and philosophical, others more practical.

We don't have much time, so I go for *very* practical with a few examples.

## Classes by example: the class of pets

A class describes data ("attributes") and behavior ("methods") of objects that are "instances" of it:

In [1]:
class Pet:
    def __init__(self, name):
        self.name = name
        self.hunger = 0
        self.age = 0
        
    def advance_time(self):
        self.age += 1
        self.hunger += 1
        
    def feed(self):
        self.hunger = 0
        
    def hungrier_than(self, other_pet):
        return self.hunger > other_pet.hunger

my_degu = Pet(name="Vitali")
my_other_degu = Pet("Wladimir")

This is how we can read the above code:

* `my_degu` and `my_other_degu` are objects, and instances of `Pet` (short: "they are pets").
* `Pet` can be understood in several ways: for example as "the concept of pet", or "the entirety of pets", or "the data type pet".
* `Pet` describes some properties and state of a pet (through the attributes `name`, `age`, `hunger`), what happens when a pet is fed (through the method `feed()`), what happens when time passes (through the method `advance_time()`), and how to compare a pet to another pet in some way (through the method `hungrier_than()`).
* `__init__` is a special method: it is called when a new Pet is created. This method is called "the constructor" of the class.
* Every method's first parameter is a reference to the object itself. By convention, it is called "self".

Let's do something with the pets:

In [2]:
for _ in range(10):
    my_degu.advance_time()
    my_other_degu.advance_time()

my_degu.feed()
print(my_other_degu.hungrier_than(my_degu))
print(my_degu.name, my_degu.age)

True
Vitali 10


You see that you can use `my_degu` and `my_other_degu` like you have used any other objects before: by accessing attributes and methods usig "`.`", and by passing objects to functions as a value.

## Class attributes and class methods

A class can not only describe data and behavior of its instances, but also of itself ("the entirety of pets", or "the concept of pets").

We redefine the Pet class a bit:

In [3]:
class Pet:
    population = set()
    
    def __init__(self, name):
        self.name = name
        self.hunger = 0
        self.age = 0
        self.__class__.population.add(self)
        
    def die(self):
        print("{} dies :(".format(self.name))
        self.__class__.population.remove(self)
        
    def is_alive(self):
        return self in self.__class__.population
        
    @classmethod
    def advance_time(cls):
        for individual in cls.population:
            individual.age += 1
            individual.hunger += 1
        
    def feed(self):
        self.hunger = 0
        
    def hungrier_than(self, other_pet):
        return self.hunger > other_pet.hunger

my_degu = Pet(name="Vitali")
my_other_degu = Pet("Wladimir")

This is how we can read what is new and exciting about our redefined class:

* `population` is a class attribute, i.e., it is defined on the class. `Pet.population` is the set of all pets. When a pet is created, it is added to the Pet population. When it `die()`s (a method we added), it is removed from the Pet population. For a given pet, we can check whether it's alive by calling `is_alive()`.
* Unlike `population`, the other attributes are instance attributes. `Pet.name` does not exist, but `my_other_degu.name` does.
* In instance methods like `__init__()` or `die()`, `self.__class__` refers to the class of self. We could just as well write `Pet` here, but it's better not to (mor on that later if we have time).
* `advance_time` is now a "class method". Its first parameter is not an instance, but the class itself. The parameter is usually called "cls".

In [4]:
for _ in range(10):
    Pet.advance_time()

my_degu.feed()
print(my_other_degu.hungrier_than(my_degu))
print(my_degu.name, my_degu.age)
print("There are {} pets.".format(len(Pet.population)))

my_degu.die()
print("There are {} pets.".format(len(Pet.population)))
my_other_degu.die()
print("There are {} pets.".format(len(Pet.population)))

True
Vitali 10
There are 2 pets.
Vitali dies :(
There are 1 pets.
Wladimir dies :(
There are 0 pets.


The only big change in the usage example is that instead of having to "advance time" for each pet individually, we now advance time for all pets in existence through our new class method.

## In Python, a class is an object

In Python, almost everything is an object. So is a class:

In [5]:
print(isinstance(Pet, object))
print(isinstance(my_degu, object))

True
True


This might look a bit confusing at first, but it really isn't. We have already used object semantics on `Pet`, and you probably did not even blink at it. The use of "`.`" to address an attribute, for instance, or the fact that we called a class method on the `Pet` class *just like* we called an instance method on a pet.

All you need to know is that you can shape a class and interact with a class in the same way as with any object. Like an object, a class has attributes holding data, and methods that you can call.

## Subclasses: the class of fishes

One thing you can do with classes is to *specialize* them by defining subclasses. For example, we can define a special kind of pet:

In [6]:
class Fish(Pet):
    def __init__(self, name, color):
        self.color = color
        super().__init__(name)
        
    def meet(self, other_fish):
        if not isinstance(other_fish, Fish):
            return
        if self.color != other_fish.color:
            if self.hungrier_than(other_fish):
                self.feed()
                other_fish.die()
            elif other_fish.hungrier_than(self):
                other_fish.feed()
                self.die()

Here's the rundown of what we did and what it means:

* `Fish` is a "subclass" of `Pet`. This means that "Fish is a Pet". Technically, it means that `Fish` "inherits" all attributes and methods from `Pet`. You'll soon see what that means exactly.
    * Nomenclature that is good to know: `Pet` is a "superclass" or "ancestor" of `Fish`. Since `Fish` is directly "derived from" `Pet`, `Pet` is also a "parent" or "base class" of `Fish`.
* `Fish.__init__()` looks a little different than `Pet.__init__()`. There is an extra parameter, and `Fish.__init__()` *calls* `Pet.__init__()` through `super()`.
* `super()` is the way to call a method in the parent class. It is a common pattern that a method that is "overridden" in a subclass (i.e. it exists in both subclass and superclass) calls `super()`. When `fish.f()` does the same as `pet.f()`, but..., then `fish.f()` probably calls `super().f()`.
* A fish can meet another fish, with predictable results.

Some code with fishes:

In [7]:
sushi = Fish(name="sushi", color="silver")
sharkie = Fish("sharkie", "blue")

print("There are {} pets.".format(len(Pet.population)))

print("first meet")
sushi.meet(sharkie)
print("time passes, and we throw in a bit of food regularly")
import random
for _ in range(11):
    Pet.advance_time()
    random.choice((sushi, sharkie)).feed()
print("second meet")
sushi.meet(sharkie)

There are 2 pets.
first meet
time passes, and we throw in a bit of food regularly
second meet
sushi dies :(


### Subclasses inherit attributes and methods from their ancestors

`Fish` "inherits" all attributes and methods from `Pet`, which in turn inherits everything from `object` because we haven't specified anything else. That means that all attributes and methods of `Pet` and `object` are visible in `Fish`, unless `Fish` makes an own attribute or method of the same name. That means that for example `Fish.population` is the same as `Pet.population`. "The same as" as in "literally the same thing"!

In [8]:
# id(obj) returns the memory address of obj
print(id(Fish.population), id(Pet.population))
print(id(Fish.population) == id(Pet.population))

140394031644232 140394031644232
True


Here's how that extends to methods (beware... technicalities ahead):

In [9]:
print(id(Fish.feed), id(Pet.feed))
print(id(Fish.feed) == id(Pet.feed))

140394030832496 140394030832496
True


Wait, wasn't `feed()` an instance method?

Yes, but it is *defined* and visible in the class, simply because we made `def feed(self)` in the scope of `class Pet`. However, Python *binds* it to an object when it is called on an object. Check this:

In [10]:
nemo = Fish("nemo", "orange-ish")

nemo.hunger = 10
nemo.feed()
print(nemo.hunger)

nemo.hunger = 10
Fish.feed(nemo)
print(nemo.hunger)

0
0


`nemo.feed()` and `Fish.feed(nemo)` do the same! That is because the first form is derived from the second form (more on that specific bit later).

Now you understand why every method inside a class has "`self`" as its first parameter, and why it is a good convention to name that parameter "`self`".

### Subclasses may override attributes and methods

`Fish` defines its own `__init__()` method. We say that `Fish` "overrides" `__init__()`. Consequently, these methods are not the same in `Fish` and `Pet`:

In [11]:
print(id(Fish.__init__), id(Pet.__init__))
print(id(Fish.__init__) == id(Pet.__init__))

140394129310856 140394030831952
False


## Objects inherit attributes from their class

Classes inherit and can override attributes from their parent classes.

Objects inherit and can override attributes from their class.

In [12]:
class A:
    def f(self):
        return self.__class__.__name__

class B(A):
    attr = 0

b = B()

print( B.f(b) ) # B inherits f from A
print( id(B.attr) == id(b.attr) ) # b inherits attr from B
b.attr = 1 # now b overrides attr
print( id(B.attr) == id(b.attr) )
print( B.attr, b.attr )

B
True
False
0 1


Fun fact about methods: instance methods just override the methods they inherit from the class. Remember:

In [13]:
B.f(b) == b.f() # b.f is just B.f "bound to b"

True

Effectively, the Python interpreter does something like this in `B.__init__()`:

```
self.f = functools.partial(B.f, self)
```

... oh, just for fun and as a little reminder, nothing stops us from setting an object's attribute to something else:

In [14]:
b2 = B()
print( b2.f() )
b2.f = lambda: "foo"
print( b2.f() )

B
foo


DON'T DO THIS AT HOME.

## Attribute lookup in objects and classes

When the Python interpreter finds `some_obj.some_attr`, this is the order of objects in which it looks after `some_attr`:

1. `some_obj`
2. `some_obj.__class__`

When the Python interpreter finds `SomeClass.some_attr`, this is the order of objects in which it looks after `some_attr`:

1. `SomeClass`
2. ancestors of `SomeClass` in [C3 linearization](https://en.wikipedia.org/wiki/C3_linearization) order

This so-called "method resolution order" can for any class be looked up like so:

In [15]:
Fish.__mro__

(__main__.Fish, __main__.Pet, object)


## Magic methods (\__*methodname*\__)

Some methods on classes are called "magic methods". Their name usually starts and ends with two underscores.

Magic methods are methods that have a special meaning within Python. For example, `__init__` is the method that is called when an instance of a class is constructed.

There are many other magic methods, however. You can override them in your own classes. (They are defined in `object`, so you are overriding them.)

For example, `__repr__`, which is supposed to "return a canonical string representation of an object" (whatever that is). You can get this "canonical representation" for any object by calling `repr(obj)`:

In [16]:
print(repr(nemo))
print(object.__repr__(nemo))
print(nemo)

That string representation is not very readable for humans, especially when we do something like this:

In [17]:
print(Pet.population)

{<__main__.Fish object at 0x7fb0084fb208>, <__main__.Fish object at 0x7fb00855e710>}


We can make this more readable by overriding `__repr__` in Pet. We do this guerilla-style here (you might not want to do this at home):

In [18]:
def better_pet_repr(o):
    return "{} ({})".format(o.name, o.__class__.__name__)
    
Pet.__repr__ = better_pet_repr

print(Pet.population)

{nemo (Fish), sharkie (Fish)}


## Interfaces

"Interfaces" are not a formal language construct in Python like they are in other languages, for example Java.

The informal meaning of "class C implements interface X" in Python is just "C implements the methods that an X has".

Examples:

* an object `obj` is "callable" when it implements `__call__()`. (Then you can call `obj()`.)
* an object `obj` "implements the dictionary interface" when it implements at the very least `__getitem__()` and `__setitem__()`. (Then you can do `obj[some_key]` and `obj[some_key] = some_value`.)

## Duck typing

> "If it quacks like a duck and looks like a duck, it's a duck."

In [19]:
class A:
    def quack(self):
        return "quackfoo"
    
class B:
    quack = lambda self: "quackbar" # saved a linebreak, woohoo!
    
def make_em_quack(*objs):
    for o in objs:
        print(o.quack())
    
make_em_quack(A(), B())

quackfoo
quackbar


The classes `A` and `B` are not related. They just both happen to have the method `quack()`. Which is all the function `make_em_quack()` needs. To that function, both `A` and `B` are sufficiently like a duck.

# TODO
    
* collections.abc: quickly make a subclass of one of the containers there
* end with reference to "stop writing classes"

### When classes?

* when you deal with "lots of entities of mutable data, with functions dealing with such entities"